## Step 1: Preprocess the Data

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("Resources/charity_data.csv")
app_df.head()

2023-03-01 16:40:02.196159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
type(app_df)

pandas.core.frame.DataFrame

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df.drop(['EIN', 'NAME'], axis=1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in first five columns and save as variable 
count = app_df[['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION']].nunique()
count

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
dtype: int64

In [5]:
# Determine the number of unique values in the second five columns and save as variable 
count2 = app_df[['STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL']].nunique()
count2

STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# For columns w/ more than 10 unique values, determine the # of data points for each value
app_vc=app_df.APPLICATION_TYPE.value_counts()
app_vc

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# # code below needed to prevent PANDAS from truncating output 
# >>> pd.set_option("display.max_rows", None)

In [8]:
class_vc=app_df.CLASSIFICATION.value_counts()
class_vc

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [12]:
# ask_vc=app_df.ASK_AMT.value_counts()
# ask_vc

In [10]:
# Look at APPLICATION_TYPE value counts for binning
binning = app_df.APPLICATION_TYPE.unique()
binning

array(['T10', 'T3', 'T5', 'T7', 'T4', 'T6', 'T2', 'T9', 'T19', 'T8',
       'T13', 'T12', 'T29', 'T25', 'T14', 'T17', 'T15'], dtype=object)

In [11]:
# # check the data types of each column in order to apply cutoff, two cells below this * Come back to this cell 
# app_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_vc[app_vc<800].index)
application_types_to_replace

['T8',
 'T7',
 'T10',
 'T9',
 'T13',
 'T12',
 'T2',
 'T25',
 'T14',
 'T29',
 'T15',
 'T17']

In [14]:
# pick a cutoff point to bin "rare" categorical variables together in a new value
for app in application_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# You may find it helpful to look at CLASSIFICATION value counts >1
binning_vc = app_df.CLASSIFICATION.value_counts()
binning_vc

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [21]:
# You may find it helpful to look at CLASSIFICATION value counts >1
binning_cutoff = binning_vc[binning_vc>1]
binning_cutoff

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list (binning_cutoff[binning_cutoff<800].index)
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256']

In [23]:
# pick a cutoff point to bin "rare" categorical variables together in a new value
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
numerical_df = pd.get_dummies(app_df,dtype=float)
numerical_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
# Split our preprocessed data into our target array
y = numerical_df['IS_SUCCESSFUL'].values
y

# Split the preprocessed data into our features array
X = numerical_df.drop('IS_SUCCESSFUL', axis=1).values
X

#Use arrays and train_test_split function to split data into training & testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [26]:
# Create a StandardScaler instances to Scale the training and testing features datasets 
scaler = StandardScaler()

# Fit the StandardScaler to the training data 
X_scaler = scaler.fit(X_train)

# Scale the data using transform function
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Step 2: Compile, Train and Evaluate the Model using TensorFlow 

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=7
hidden_nodes_layer2=14
hidden_nodes_layer3=21
nn = tf.keras.models.Sequential()


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

2023-03-01 17:11:44.617213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 137452    
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 137,579
Trainable params: 137,579
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 13s 16ms/step - loss: 0.5597 - accuracy: 0.7213 - val_loss: 0.4776 - val_accuracy: 0.7769
Epoch 2/100
684/684 [==============================] - 6s 9ms/step - loss: 0.3547 - accuracy: 0.8401 - val_loss: 0.5131 - val_accuracy: 0.7834
Epoch 3/100
684/684 [==============================] - 5s 8ms/step - loss: 0.1777 - accuracy: 0.9321 - val_loss: 0.6611 - val_accuracy: 0.6608
Epoch 4/100
684/684 [==============================] - 5s 8ms/step - loss: 0.1166 - accuracy: 0.9512 - val_loss: 0.6103 - val_accuracy: 0.7803
Epoch 5/100
684/684 [==============================] - 8s 11ms/step - loss: 0.1055 - accuracy: 0.9539 - val_loss: 0.6833 - val_accuracy: 0.7813
Epoch 6/100
684/684 [==============================] - 7s 11ms/step - loss: 0.1016 - accuracy: 0.9567 - val_loss: 1.0205 - val_accuracy: 0.7828
Epoch 7/100
684/684 [==============================] - 6s 9ms/step - loss: 0.0990 - accuracy: 0.9573 - val_loss: 1.0270 - val_accuracy: 0.

Epoch 58/100
684/684 [==============================] - 4s 6ms/step - loss: 0.0833 - accuracy: 0.9644 - val_loss: 5.4082 - val_accuracy: 0.5877
Epoch 59/100
684/684 [==============================] - 4s 5ms/step - loss: 0.0826 - accuracy: 0.9651 - val_loss: 5.5138 - val_accuracy: 0.5869
Epoch 60/100
684/684 [==============================] - 4s 5ms/step - loss: 0.0829 - accuracy: 0.9648 - val_loss: 5.0814 - val_accuracy: 0.5898
Epoch 61/100
684/684 [==============================] - 4s 5ms/step - loss: 0.0827 - accuracy: 0.9648 - val_loss: 6.2829 - val_accuracy: 0.5872
Epoch 62/100
684/684 [==============================] - 4s 6ms/step - loss: 0.0829 - accuracy: 0.9650 - val_loss: 6.1550 - val_accuracy: 0.5869
Epoch 63/100
684/684 [==============================] - 4s 6ms/step - loss: 0.0828 - accuracy: 0.9645 - val_loss: 7.2597 - val_accuracy: 0.5851
Epoch 64/100
684/684 [==============================] - 4s 6ms/step - loss: 0.0825 - accuracy: 0.9651 - val_loss: 6.3560 - val_accuracy:

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.6760 - accuracy: 0.6168 - 2s/epoch - 6ms/step
Loss: 0.6759758591651917, Accuracy: 0.6167929768562317


In [32]:
# # Export our model to HDF5 file
# from google.colab import files

# nn.save('/content/Model_1.h5')
# files.download('/content/Model_1.h5')